## Wine SPA Dataset

In [ ]:
#Vista previa de 10 datos aleatorios
df_wine_spa.sample(10)

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
6700,Remirez de Ganuza,Rioja Blanco,2017,4.2,421,Espana,Rioja,29.15,Rioja White,3.0,3.0
3755,Losada,Altos de Losada,2018,4.2,415,Espana,Bierzo,17.90,Mencia,3.0,3.0
6947,Mustiguillo,Finca Terrerazo,2017,4.2,390,Espana,El Terrerazo,24.45,Red,4.0,3.0
2103,Sierra Cantabria,Coleccion Privada,2012,4.2,420,Espana,Rioja,28.30,Rioja Red,4.0,3.0
4760,Campillo,Gran Reserva Rioja,2012,4.2,410,Espana,Rioja,22.99,Rioja Red,4.0,3.0
2318,Contino,Rioja Graciano,2011,4.2,395,Espana,Rioja,61.94,Rioja Red,4.0,3.0
7284,Bodegas La Horra,Corimbo I,2015,4.2,412,Espana,Ribera del Duero,37.90,Ribera Del Duero Red,5.0,3.0
1551,Marques de Vargas,Seleccion Privada,2015,4.3,79,Espana,Rioja,54.50,Rioja Red,4.0,3.0
2829,Imperial,Rioja Reserva,2004,4.2,417,Espana,Rioja,45.00,Rioja Red,4.0,3.0
3425,La Vicalanda,Reserva,2015,4.2,420,Espana,Rioja,17.50,Rioja Red,4.0,3.0
